Cell 1

In [ ]:
import numpy as np
import tkinter as tk
import copy
import pickle as pickle 
import random as rn
import time
import matplotlib.pyplot as plt
import seaborn as sns

Cell 2

In [ ]:
#pip install numpy tk pickle-mixin matplotlib seaborn

Cell 3

In [ ]:
# Variable initialization
grid_size = 3
N_episodes = 1000
decay = 1             #decay = 1 for epsilon decay, decay = 0 for greedy epsilon
epsilon = 0.5         # epsilon in the case where decay = 0
sym = 1               # sym = 1 to activate symmetry incorporation, sym = 0 otherwise
epsilon_init = 1      #initialize initial epsilon (1 for complete random moves)
epsilon_end = 0      #initialize initial epsilon (0 for complete greedy moves)

Cell 4

In [ ]:
class Game:
    def __init__(self, master, player1, player2,N_episodes=0,decay =1,epsilon=0,epsilon_init=1,epsilon_end=0,sym =0, Q_learn=None, Q={}, alpha=0.3, gamma=0.9):
        frame = tk.Frame()
        frame.grid()
        self.master = master
        master.title("Tic Tac Toe")
        self.player1 = player1
        self.player2 = player2
        self.current_player = player1
        self.other_player = player2
        self.empty_text = ""
        self.board = Board()
        self.nsteps=0
        self.epsilon = epsilon
        self.epsilon_init = epsilon_init
        self.epsilon_end = epsilon_end
        self.decay = decay
        self.win_X_count = 0
        self.win_X_100 = []
        self.win_O_count = 0
        self.win_O_100 = []
        self.draw_count = 0
        self.draw_100 = []
        self.total = 0
        self.total_100 = []
        self.change_list = []    
        if grid_size == 3:
            self.mask = np.array([[(0,0),((0,1)),(0,2)],[(1,0),(1,1),(1,2)],[(2,0),(2,1),(2,2)]])
        elif grid_size == 4:
            self.mask = np.array([[(0,0),((0,1)),(0,2),(0,3)],[(1,0),(1,1),(1,2),(1,3)],[(2,0),(2,1),(2,2),(2,3)],
                              [(3,0),(3,1),(3,2),(3,3)]])
        self.mask_rot90cc = np.rot90(self.mask,1)
        self.mask_rot180cc = np.rot90(self.mask,2)
        self.mask_rot270cc = np.rot90(self.mask,3)
        self.mask_flipx = np.flip(self.mask,0)
        self.mask_flipy = np.flip(self.mask,1)
        self.learn = sym

        # creating the buttons for the tic tac toe games
        self.buttons = [[None for _ in range(grid_size)] for _ in range(grid_size)]
        for i in range(grid_size):
            for j in range(grid_size):
                self.buttons[i][j] = tk.Button(frame, height=grid_size, width=grid_size, text=self.empty_text, command=lambda i=i, j=j: self.callback(self.buttons[i][j]))
                self.buttons[i][j].grid(row=i, column=j)
        #creating reset button
        self.reset_button = tk.Button(text="Reset", command=self.reset)
        self.reset_button.grid(row=grid_size)

        self.Q_learn = Q_learn
        if self.Q_learn:
            self.Q = Q
            self.alpha = alpha          # Learning rate
            self.gamma = gamma          # Discount rate
            self.share_Q_with_players()
        
    @property
    def Q_learn(self):
        if self._Q_learn is not None:    
            return self._Q_learn
        if isinstance(self.player1, QPlayer) or isinstance(self.player2, QPlayer):
            return True

    @Q_learn.setter
    def Q_learn(self, _Q_learn):
        self._Q_learn = _Q_learn
        
    def share_Q_with_players(self):             # The action value table Q is shared with the QPlayers to help them make their move decisions
        if isinstance(self.player1, QPlayer):
            self.player1.Q = self.Q
        if isinstance(self.player2, QPlayer):
            self.player2.Q = self.Q
    
    #
    def callback(self, button):
        if self.board.over():
            pass                # Do nothing if the game is already over
        else:       
            # if the current player is the human player and the other player is the computer player
            if isinstance(self.current_player, HumanPlayer) and isinstance(self.other_player, ComputerPlayer):
                computer_player = self.other_player
                if self.empty(button):
                    human_move = self.get_move(button)
                    self.handle_move(human_move)
                    if not self.board.over():               # Trigger the computer's next move
                        computer_move = computer_player.get_move(self.board)
                        self.handle_move(computer_move)

    def empty(self, button):
        return button["text"] == self.empty_text

    def get_move(self, button):
        info = button.grid_info()
        # Get move coordinates from the button's metadata
        move = (int(info["row"]), int(info["column"]))
        return move

    def handle_move(self, move):
        if self.Q_learn: 
            self.learn_Q(move)
        i, j = move         # Get row and column number of the corresponding button
        self.buttons[i][j].configure(text=self.current_player.mark)     # Change the label on the button to the current player's mark
        self.board.place_mark(move, self.current_player.mark)           # Update the board
        if self.board.over():
            self.declare_outcome()
        else:
            self.switch_players()

    def declare_outcome(self):
        self.total = self.total+1
        if self.board.winner() is None:
            self.draw_count = self.draw_count+1
            print("Cat's game.")
        else:
            if self.current_player.mark == "X":
                self.win_X_count = self.win_X_count+1
            elif self.current_player.mark == "O":
                self.win_O_count = self.win_O_count+1    
            print(("The game is over. The player with mark {mark} won!".format(mark=self.current_player.mark)))
            
        if (self.nsteps+1)%100 == 0 and self.nsteps>0:
            self.total_100 = self.total_100 + [self.total]
            self.draw_100 = self.draw_100 + [self.draw_count]
            self.win_X_100 = self.win_X_100 + [self.win_X_count]
            self.win_O_100 = self.win_O_100 + [self.win_O_count]

    def reset(self):
#         print("Resetting...")
        for i in range(grid_size):
            for j in range(grid_size):
                self.buttons[i][j].configure(text=self.empty_text)
        self.board = Board(grid=np.ones((grid_size,grid_size))*np.nan)
        self.current_player = self.player1
        self.other_player = self.player2
#         np.random.seed(seed=0)      # Set the random seed to zero to see the Q-learning 'in action' or for debugging purposes
        self.play(nsteps=self.nsteps)

    def switch_players(self):
        if self.current_player == self.player1:
            self.current_player = self.player2
            self.other_player = self.player1
        else:
            self.current_player = self.player1
            self.other_player = self.player2

    def play(self,nsteps=0):
        if self.decay ==1:
            self.nsteps=nsteps
            r = max((N_episodes-self.nsteps)/N_episodes,0)
            epsilon = (self.epsilon_init-self.epsilon_end)*r+self.epsilon_end
        else: epsilon = self.epsilon
        self.total = 0    
        self.change_total = 0
        if isinstance(self.player1, HumanPlayer) and isinstance(self.player2, ComputerPlayer):
            pass
        # Make the two computer players play against each other without button presses
        elif isinstance(self.player1, ComputerPlayer) and isinstance(self.player2, ComputerPlayer):
            if isinstance (self.player1, QPlayer) and isinstance(self.player2, QPlayer):
                self.player1.epsilon = epsilon
                self.player2.epsilon = epsilon
                while not self.board.over():
                    self.play_turn()
            elif isinstance (self.player1, QPlayer) and isinstance(self.player2, RandomPlayer):
                self.player1.epsilon = epsilon
                self.player1.randomMoveNo = 0
                self.player1.greedyMoveNo = 0
                while not self.board.over():
                    self.play_turn()    
        
        if self.total!=0:
#             print(self.change_total,self.total)
            self.change_list = self.change_list + [self.change_total]
            


    def play_turn(self):
        move = self.current_player.get_move(self.board) 
        self.handle_move(move)
        
    # If Q-learning is toggled on, "learn_Q" should be called after receiving a move from an instance of Player 
    # and before implementing the move (using Board's "place_mark" method)
    def learn_Q(self, move): 
        state_key = QPlayer.make_and_maybe_add_key(self.board, self.current_player.mark, self.Q)
        next_board = self.board.get_next_board(move, self.current_player.mark)
        reward = next_board.give_reward()
        next_state_key = QPlayer.make_and_maybe_add_key(next_board, self.other_player.mark, self.Q)
        
            
        if next_board.over():
            expected = reward
        else:
            next_Qs = self.Q[next_state_key]             # The Q values represent the expected future reward for player X for each available move in the next state (after the move has been made)
            if self.current_player.mark == "X":
                expected = reward + (self.gamma * min(next_Qs.values()))        # If the current player is X, the next player is O, and the move with the minimum Q value should be chosen according to our "sign convention"
            elif self.current_player.mark == "O":
                expected = reward + (self.gamma * max(next_Qs.values()))        # If the current player is O, the next player is X, and the move with the maximum Q vlue should be chosen
        
        
        change = self.alpha * (expected - self.Q[state_key][move])
        self.Q[state_key][move] += change
        self.change_total = self.change_total + change
        
        
        
        if self.learn == 1:
            r = rn.random()
            state_key_dict = {}
            default_Qvalue = 0.5 +  r
            if default_Qvalue>1:
                default_Qvalue = 1
            #90 degree rotation counterclockwise
            state_key_90cc = self.board.make_key(np.rot90(self.board.grid),self.current_player.mark)
            if self.Q.get(state_key_90cc) is None:
                moves = self.board.available_moves_sym(np.rot90(self.board.grid))
                # assigns values to the Q value of the state
                self.Q[state_key_90cc] = {move: default_Qvalue for move in moves} 
            
            if self.Q.get(state_key_90cc) is not None and state_key!=state_key_90cc:
                for i in range(grid_size):
                    for j in range(grid_size):
                        if self.mask_rot90cc[i,j][0]==move[0] and self.mask_rot90cc[i,j][1]==move[1]:
                            move90cc = (i,j)         
                if state_key_90cc not in state_key_dict:
                    state_key_dict[state_key_90cc]=move90cc
#                     print("90",state_key,move,state_key_90cc,move90cc)
            
                
                
            #180 degree rotation counterclockwise
            state_key_180cc = self.board.make_key(np.rot90(self.board.grid,2),self.current_player.mark)
            if self.Q.get(state_key_180cc) is None:
                moves = self.board.available_moves_sym(np.rot90(self.board.grid,2))
                # assigns values to the Q value of the state
                self.Q[state_key_180cc] = {move: default_Qvalue for move in moves} 
            if self.Q.get(state_key_180cc) is not None and state_key!=state_key_180cc:
                for i in range(grid_size):
                    for j in range(grid_size):
                        if self.mask_rot180cc[i,j][0]==move[0] and self.mask_rot180cc[i,j][1]==move[1]:
                            move180cc = (i,j)
                if state_key_180cc not in state_key_dict:
                    state_key_dict[state_key_180cc]=move180cc
#                     print("180",state_key,move,state_key_180cc,move180cc)
                
                
            #270 degree rotation counterclockwise
            state_key_270cc = self.board.make_key(np.rot90(self.board.grid,3),self.current_player.mark)
            if self.Q.get(state_key_270cc) is None:
                moves = self.board.available_moves_sym(np.rot90(self.board.grid,3))
                # assigns values to the Q value of the state
                self.Q[state_key_270cc] = {move: default_Qvalue for move in moves} 
            if self.Q.get(state_key_270cc) is not None and state_key!=state_key_270cc:
                for i in range(grid_size):
                    for j in range(grid_size):
                        if self.mask_rot270cc[i,j][0]==move[0] and self.mask_rot270cc[i,j][1]==move[1]:
                            move270cc = (i,j)
                if state_key_270cc not in state_key_dict:
                    state_key_dict[state_key_270cc]=move270cc
#                     print("270",state_key,move,state_key_270cc,move270cc)
                
            #flipx degree rotation counterclockwise
            state_key_flipx = self.board.make_key(np.flip(self.board.grid,0),self.current_player.mark)
            if self.Q.get(state_key_flipx) is None:
                moves = self.board.available_moves_sym(np.flip(self.board.grid,0))
                # assigns values to the Q value of the state
                self.Q[state_key_flipx] = {move: default_Qvalue for move in moves} 
            
            if self.Q.get(state_key_flipx) is not None and state_key!=state_key_flipx:
                for i in range(grid_size):
                    for j in range(grid_size):
                        if self.mask_flipx[i,j][0]==move[0] and self.mask_flipx[i,j][1]==move[1]:
                            moveflipx = (i,j)
                if state_key_flipx not in state_key_dict:
                    state_key_dict[state_key_flipx]=moveflipx
#                     print("flipx",state_key,move,state_key_flipx,moveflipx)
                
                
            #flipy degree rotation counterclockwise
            state_key_flipy = self.board.make_key(np.flip(self.board.grid,1),self.current_player.mark)
            if self.Q.get(state_key_flipy) is None:
                moves = self.board.available_moves_sym(np.flip(self.board.grid,1))
                # assigns values to the Q value of the state
                self.Q[state_key_flipy] = {move: default_Qvalue for move in moves} 
            if self.Q.get(state_key_flipy) is not None and state_key!=state_key_flipy:
                for i in range(grid_size):
                    for j in range(grid_size):
                        if self.mask_flipy[i,j][0]==move[0] and self.mask_flipy[i,j][1]==move[1]:
                            moveflipy = (i,j)
                if state_key_flipy not in state_key_dict:
                    state_key_dict[state_key_flipy]=moveflipy
#                     print("flipy",state_key,move,state_key_flipy,moveflipy)
                
            for key in state_key_dict:
                change = self.alpha * (expected - self.Q[key][state_key_dict[key]])
                self.Q[key][state_key_dict[key]] += change
                
        
        
        

class Board:
    def __init__(self, grid=np.ones((grid_size,grid_size))*np.nan):
        self.grid = grid

    def winner(self):
        rows = [self.grid[i,:] for i in range(grid_size)]
        cols = [self.grid[:,j] for j in range(grid_size)]
        diag = [np.array([self.grid[i,i] for i in range(grid_size)])]
        cross_diag = [np.array([self.grid[2-i,i] for i in range(grid_size)])]
        lanes = np.concatenate((rows, cols, diag, cross_diag))      # A "lane" is defined as a row, column, diagonal, or cross-diagonal

        any_lane = lambda x: any([np.array_equal(lane, x) for lane in lanes])   # Returns true if any lane is equal to the input argument "x"
        if any_lane(np.ones(grid_size)):
            return "X"
        elif any_lane(np.zeros(grid_size)):
            return "O"
        
    # The game is over if there is a winner or if no squares remain empty (cat's game)
    def over(self):             
        return (not np.any(np.isnan(self.grid))) or (self.winner() is not None)

    def place_mark(self, move, mark):       # Place a mark on the board
        num = Board.mark2num(mark)
        self.grid[tuple(move)] = num

    @staticmethod
    def mark2num(mark):         # Convert's a player's mark to a number to be inserted in the Numpy array representing the board. The mark must be either "X" or "O".
        d = {"X": 1, "O": 0}
        return d[mark]

    def available_moves(self):
        # returns the coordinates with nan values in the grid
        return [(i,j) for i in range(grid_size) for j in range(grid_size) if np.isnan(self.grid[i][j])]

    def available_moves_sym(self,grid):
        # returns the coordinates with nan values in the grid
        return [(i,j) for i in range(grid_size) for j in range(grid_size) if np.isnan(grid[i][j])]


    def get_next_board(self, move, mark):
        next_board = copy.deepcopy(self)
        next_board.place_mark(move, mark)
        return next_board
    # For Q-learning, returns a 10-character string representing the state of the board and the player whose turn it is
    def make_key(self,grid, mark):          
        fill_value = grid_size**2
        filled_grid = copy.deepcopy(grid)  #https://www.geeksforgeeks.org/copy-python-deep-copy-shallow-copy/
        np.place(filled_grid, np.isnan(filled_grid), fill_value) #https://www.geeksforgeeks.org/numpy-place-python/
        return "".join(map(str, (list(map(int, filled_grid.flatten()))))) + mark 

    def give_reward(self):                          # Assign a reward for the player with mark X in the current board position.
        if self.over():
            if self.winner() is not None:
                if self.winner() == "X":
                    return 1.0                      # Player X won -> positive reward
                elif self.winner() == "O":
                    return -1.0                     # Player O won -> negative reward
            else:
                return 0.5                          # A smaller positive reward for cat's game
        else:
            return 0.0                              # No reward if the game is not yet finished

class Player(object):
    def __init__(self, mark):
        self.mark = mark

    @property
    def opponent_mark(self):
        if self.mark == 'X':
            return 'O'
        elif self.mark == 'O':
            return 'X'
        else:
            print("The player's mark must be either 'X' or 'O'.")

class HumanPlayer(Player):
    pass
class ComputerPlayer(Player):
    pass
class RandomPlayer(ComputerPlayer):
    @staticmethod
    def get_move(board):
        moves = board.available_moves()
        # If "moves" is not an empty list (as it would be if cat's game were reached)
        if moves:   
            # Apply random selection to the index, as otherwise it will be seen as a 2D array
            return moves[np.random.choice(len(moves))]    
        

    
class QPlayer(ComputerPlayer):
    #initialization of a Qplayer
    def __init__(self, mark, Q={}):
        super(QPlayer, self).__init__(mark=mark)
        self.Q = Q
        self.randomMoveNo = 0
        self.greedyMoveNo = 0
    
    def get_move(self, board):
        # With probability epsilon, choose a move at random ("epsilon-greedy" exploration)
        if np.random.uniform() < epsilon:
            self.randomMoveNo = self.randomMoveNo + 1
            moves = board.available_moves()
            move = moves[np.random.choice(len(moves))] 
            return move
        else:
            self.greedyMoveNo = self.greedyMoveNo + 1
            state_key = QPlayer.make_and_maybe_add_key(board,self.mark, self.Q) # gets the state of the game
            Qs = self.Q[state_key] # gets the q values of the state_key obtained
            if self.mark == "X":
                return QPlayer.stochastic_argminmax(Qs, max)
            elif self.mark == "O":
                return QPlayer.stochastic_argminmax(Qs, min)

    @staticmethod
    # Make a dictionary key for the current state (board + player turn) and if Q does not yet have it, add it to Q
    def make_and_maybe_add_key(board, mark, Q):  
        r = rn.random()
        # Encourages exploration
        default_Qvalue = 0.5 +  r
        if default_Qvalue>1:
            default_Qvalue=1
        state_key = board.make_key(board.grid,mark) # gets the state of the board (10 element str) and the player playing 
        
        if Q.get(state_key) is None:
                        moves = board.available_moves()
                        # assigns values to the Q value of the state
                        Q[state_key] = {move: default_Qvalue for move in moves}    # The available moves in each state are initially given a default value of zero
                    
        return state_key

    
    
    @staticmethod
    # Determines either the argmin or argmax of the array Qs such that if there are 'ties', one is chosen at random
    def stochastic_argminmax(Qs, min_or_max):       
        min_or_maxQ = min_or_max(list(Qs.values()))
        # If there is more than one move corresponding to the maximum Q-value, choose one at random
        if list(Qs.values()).count(min_or_maxQ) > 1:      
            best_options = [move for move in list(Qs.keys()) if Qs[move] == min_or_maxQ]
            move = best_options[np.random.choice(len(best_options))]
        else:
            move = min_or_max(Qs, key=Qs.get)
        return move





Cell 5

In [ ]:


root = tk.Tk()
player1 = QPlayer(mark="X")
player2 = QPlayer(mark="O")

game = Game(root, player1, player2,N_episodes = N_episodes,decay = 1,sym=1,epsilon=epsilon)
NoOfrandom = []
NoOfgreedy =[]
total = []
for episodes in range(N_episodes):
    print("Episode:",episodes)
    game.play(nsteps=episodes)
    game.reset()
Q = game.Q
filename = "Q_epsilon_09_Nepisodes_{} (3by3).p".format(N_episodes)
pickle.dump(Q, open(filename, "wb"))


Cell 6


In [ ]:
Q = pickle.load(open("Q_epsilon_09_Nepisodes_"+str(N_episodes)+" (3by3).p", "rb"))

root = tk.Tk()
player1 = HumanPlayer(mark="X")
player2 = QPlayer(mark="O")

game = Game(root, player1, player2, Q=Q,decay=0)

game.play()
root.mainloop()